In [1]:
import multiprocessing
import threading
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------- Definitions --------------

def cpu_stress(cores, duration, output_box):
    def cpu_load():
        while not stop_event.is_set():
            pass  # Busy wait
    stop_event = threading.Event()
    processes = []
    try:
        output_box.append_stdout(f"Starting {cores} CPU stress processes...\n")
        for _ in range(cores):
            p = multiprocessing.Process(target=cpu_load)
            p.start()
            processes.append(p)
        output_box.append_stdout(f"Stressing CPU for {duration} seconds...\n")
        time.sleep(duration)
    finally:
        stop_event.set()
        for p in processes:
            p.terminate()
        output_box.append_stdout("Stopped CPU stress test.\n\n")

def ram_stress(mb, duration, output_box):
    big_data = []
    try:
        output_box.append_stdout(f"Allocating ~{mb} MB RAM...\n")
        for _ in range(mb):
            big_data.append(bytearray(1024 * 1024))  # Allocate 1MB
        output_box.append_stdout(f"RAM allocation done. Holding for {duration} seconds...\n")
        time.sleep(duration)
    except MemoryError:
        output_box.append_stdout("MemoryError: Could not allocate requested memory.\n")
    finally:
        del big_data
        output_box.append_stdout("Released memory.\n\n")

def combined_stress(cores, mb, duration, output_box):
    # Start both CPU and RAM stress in threads so both run concurrently
    cpu_thread = threading.Thread(target=cpu_stress, args=(cores, duration, output_box))
    ram_thread = threading.Thread(target=ram_stress, args=(mb, duration, output_box))
    output_box.append_stdout("Running combined CPU and RAM stress...\n")
    cpu_thread.start()
    ram_thread.start()
    cpu_thread.join()
    ram_thread.join()
    output_box.append_stdout("Combined stress test complete.\n\n")

# ----------- Widgets & UI -------------

warning = widgets.HTML("<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—"
                      "setting too high may crash your environment or cause instability.")

cpu_count = multiprocessing.cpu_count()
cpu_input = widgets.BoundedIntText(value=min(2, cpu_count), min=1, max=cpu_count,
                                   description="CPU cores:")
ram_input = widgets.BoundedIntText(value=500, min=10, max=20000, step=10,
                                   description="RAM (MB):")
dur_input = widgets.BoundedIntText(value=30, min=5, max=600, step=5,
                                   description="Duration (s):")

cpu_button = widgets.Button(description="Run CPU Stress Test", button_style="warning")
ram_button = widgets.Button(description="Run RAM Stress Test", button_style="warning")
both_button = widgets.Button(description="Run BOTH", button_style="danger")

output_box = widgets.Output()

def on_cpu_clicked(b):
    with output_box:
        clear_output()
        cpu_stress(cpu_input.value, dur_input.value, output_box)

def on_ram_clicked(b):
    with output_box:
        clear_output()
        ram_stress(ram_input.value, dur_input.value, output_box)

def on_both_clicked(b):
    with output_box:
        clear_output()
        combined_stress(cpu_input.value, ram_input.value, dur_input.value, output_box)

cpu_button.on_click(on_cpu_clicked)
ram_button.on_click(on_ram_clicked)
both_button.on_click(on_both_clicked)

controls = widgets.HBox([cpu_input, ram_input, dur_input])
buttons = widgets.HBox([cpu_button, ram_button, both_button])

display(warning, controls, buttons, output_box)

HTML(value="<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—setting…

Output()